In [4]:
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


In [5]:
from stackable_pipe import pipe, slot, ring, socket
from solid import cylinder, translate, rotate


def spin_around_tower(obj, count=4, alpha=45, h=160):
    angles = range(0, 360, int(360/count))
    # drehen und dann kippen
    return translate((0, 0, h/2))([rotate((alpha, 0, z))(obj) for z in angles])


def tower_socket(d=60, w=4):
    r = d/2
    base = rotate((0, 0, 15))(socket(d=d, w=w)) + ring(d1=d, h=d+2*w, w=w/2)
    transition = ring(r2=r+w*3, r1=r, h=w*3, w=w*4) - cylinder(h=w*3, d=d)

    return base  # + transition


def aero_tower(w=4, h=160, d=160, pipe_count=5, alpha=45):
    tower = pipe(d=d, h=h, w=w)
    outline = cylinder(d=d, h=d, segments=60)
    holes = spin_around_tower(
        cylinder(d=60, h=h, segments=60), count=pipe_count, alpha=alpha)
    sockets = spin_around_tower(
        translate((0, 0, d/2))(tower_socket(w=w)), h=d, count=pipe_count, alpha=alpha)

    holes = translate((0, 0, -h/2))(holes)
    sockets = translate((0, 0, -h/2))(sockets)

    aero_tower = tower - holes
    aero_tower += sockets
    aero_tower *= outline
    return aero_tower


render(aero_tower(), outfile='./outfiles/aero_tower_module.stl')


In [12]:
from stackable_pipe import pipe, slot, ring, socket, connection
from solid import cylinder, translate, rotate, cube
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


def spin_around_tower(obj, count=4, alpha=45, h=160):
    angles = range(180, 540, int(360/count))
    # drehen und dann kippen
    return translate((0, 0, h/2))([rotate((alpha, 0, z))(obj) for z in angles])


def tower_socket(d=60, w=4):
    r = d/2
    c = d
    td=2
    cutout = cube(c)\
        - translate((c/2, 0, 0))(cylinder(r=c/2, h=c, segments=100))\
        - translate((c/2, c, 0))(cylinder(r=c/2, h=c, segments=100))\
        - translate((c/2, 0, 0))(cube(c))

    cutout = translate((c/2, -c/2, 0))(rotate((0, -90, 0))(cutout))
    cutout = rotate(90)(cutout)
    cutout = translate((0, c/2, 0))(cutout)
    (adapter,socket) = connection(d=d, h=6*td,  tooth_depth=td, tooth_height=2*td,)
    base = ring(d1=d, h=d+2*w, w=w/2)
    return base + socket - cutout


def test(w=2, h=160, d=160, pipe_count=1, alpha=45):
    gap = 0.2
    td = 2
    delta = w + td + gap 
    tower = ring(d1=d, h=h-20+delta, w=w)
    outline = cylinder(d=d, h=d, segments=100)
    holes = spin_around_tower(
        cylinder(d=60, h=h, segments=100), count=pipe_count, alpha=alpha)
    sockets = spin_around_tower(
        translate((0, 0, h/2))(tower_socket(w=2*w)), h=d, count=pipe_count, alpha=alpha)

    holes = translate((0, 0, -h/2))(holes)
    sockets = translate((0, 0, -h/2))(sockets)
    (adapter, socket) = connection(d=d, h=20,
                                   wall=w, tooth_depth=td, tooth_height=2*td, gap=gap)
    aero_tower = translate((0, 0, 20+delta))(tower)
    aero_tower = aero_tower - holes + sockets
    aero_tower += adapter + \
        translate((0, 0, 20))(ring(r2=d/2, r1=d/2-delta, h=delta))

    aero_tower += translate((0, 0, h-20))(socket)
    aero_tower *= outline
    return aero_tower


render(test(), outfile='./outfiles/new_tower.stl')


EXPORT-WARNING: Exported object may not be a valid 2-manifold and may need repair
/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [6]:
from stackable_pipe import pipe, slot, ring, socket, cube
from solid import cylinder, translate, rotate


def bottom_grid(d=160, fogger_diameter=100, slots=5):
    plate = cylinder(h=2, d=d, segments=60)
    hole = translate((fogger_diameter/2, -5, 0)
                     )(cube(((d-fogger_diameter)/2, 10, 2)))

    holes = sum([rotate((0, 0, r))(hole) for r in range(0, 360, 360//slots)])

    return plate - holes


def aero_tower_fogger_inlet(w=4, d=160, fogger_height=80, fogger_diameter=100, fogger_gap=1, fog_slots=5):
    h = fogger_height + 7*w + 2

    tower = pipe(d=d, h=h, w=w)

    inlet = cylinder(segments=60, d=fogger_diameter+w, h=fogger_height + 2)\
        - cylinder(segments=60, d=fogger_diameter +
                   fogger_gap, h=fogger_height + 2)

    return tower + inlet + bottom_grid(d=d-4*w, slots=10, fogger_diameter=fogger_diameter+w)


render(aero_tower_fogger_inlet(),
       outfile='./outfiles/aero_tower_fogger_inlet.stl')


In [7]:
from stackable_pipe import pipe, slot, ring, socket, adapter
from logo import logo
from solid import cylinder, translate, rotate, cube, hull

from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


def adapter_bottom_grid(d=60, w=4):
    plate = cylinder(h=2, d=d-4*w-w/2, segments=10)
    hole = cylinder(h=2, d=6, segments=10)

    holes = sum([rotate((0, 0, r))(translate((x, 0, 0))(hole))
                for r in range(0, 360, 360//8) for x in [0, 10, 18]])

    return plate - holes


def aero_tower_plant_cup(w=4, pipe_wall=2, d=60, h=90, gap=2):
    pipe_height = 5*w
    bottom = adapter(d=d, w=w, gap=0.4)
    grid = adapter_bottom_grid(d=d, w=w)
    # Gap außen wegnehmen und innen beifügen (halbiert, da außen vom Durchmesser abgezogen wird)
    pipe = ring(d1=d-w, w=pipe_wall+gap/2, h=h-pipe_height, dz=pipe_height)
    outline = cylinder(d=d-(w+gap), h=h, segments=10)

    return (bottom + grid + pipe) * outline


l = translate((0, 0, 65))(rotate((0, 90, 0))(logo(h=40, w=30)))\
    * ring(d1=54, h=90, w=0.5)  # outline s.o.

logos = sum((rotate(a)(l) for a in [0, 120, 240]))

c = rotate(20)(aero_tower_plant_cup())
cup_with_logo = c - logos

render(logos)
render(cup_with_logo)
# render(cup_with_logo, outfile='./outfiles/aero_tower_plant_cup.stl')


In [ ]:
from solid import cylinder, sphere, translate, cube
from stackable_pipe import socket
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


def tank(size=180, wall=2, border_radius=30, socket_d=160, socket_w=4):
    top_height = size-socket_d+socket_w*3

    body = cylinder(d=size, h=size-top_height, segments=100)\
        - translate((0, 0, wall))(cylinder(d=(size-(wall*2)),
                                           h=size, segments=100))

    top = translate((0, 0, size-socket_d))(socket(d=socket_d, w=socket_w))\
        + cylinder(d1=size, d2=socket_d, h=size-socket_d, segments=100)\
        - cylinder(d1=size-(wall*2), d2=socket_d -
                   (wall*2), h=size-socket_d, segments=100)

    return body + translate((0, 0, size-top_height))(top)


render(tank(), outfile='./outfiles/aero_tower_tank.stl')


/workspace/.pip-modules/lib/python3.8/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [ ]:
from viewscad import Renderer
from solid import cube, hull,square, rotate_extrude, translate
from stackable_pipe import socket, slot

r = Renderer()
r.width = 700
r.height = 700

render = r.render



render(socket(d=160,w=4))



In [2]:
from solid import cylinder, translate, rotate
from stackable_pipe import adapter
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


def lid(d=160, wall=2, socket_w=4):
    body = adapter(d=d, w=socket_w)
    body = translate((0, 0, wall+7*socket_w))(rotate((180, 0, 0))(body))
    base = cylinder(d=d, h=wall, segments=100)

    return body + base


# render(lid(), outfile='./outfiles/aero_tower_lid.stl')

render(lid()-cylinder(d=72,h=40,segments=100), outfile='./outfiles/aero_tower_lid_w_hole.stl')

